In [1]:
from solid import *
from solid.utils import *
import viewscad

r = viewscad.Renderer(openscad_exec='C:\Program Files\OpenSCAD\openscad.exe')

In [2]:
#Gen 1
def lego(m,n): # Base with Posts
    sf=1 # Scale factor
    h=0.6 # height of block
    dh = 0.8 # Hole diameter
    
    # 4 corners
    c = translate([0.5,0.5,0])(cylinder(h = h, d = sf, segments=100)) 
    c = c + translate([m-0.5,0.5,0])(cylinder(h = h, d = sf, segments=100))
    c = c + translate([0.5,n-0.5,0])(cylinder(h = h, d = sf, segments=100))
    c = c + translate([m-0.5,n-0.5,0])(cylinder(h = h, d = sf, segments=100))
    p=hull()(c)
    
    # Holes
    for i in range(m):
        for j in range(n):
            p = p - translate([i+0.5,j+0.5,0]) (cylinder(h=h,d=dh,segments=100))
    
    return(p)

#lego(5,2)
m=int(input('m : '))
n=int(input('n : '))
brick = lego(m,n)
r.render(brick)

m : 5
n : 3


In [3]:
#Gen 1 lab
def legol(m,n): # Base with Posts
    
    h= 7 # height of block
    dh = 4.8 # Hole diameter
    sd = 3.2
    sf = dh +sd # Scale factor
    
    # 4 corners
    c = translate([0,0,0])(cylinder(h = h, d = sf, segments=100)) 
    c = c + translate([(m-1)*sf,0,0])(cylinder(h = h, d = sf, segments=100))
    c = c + translate([0,(n-1)*sf,0])(cylinder(h = h, d = sf, segments=100))
    c = c + translate([(m-1)*sf,(n-1)*sf,0])(cylinder(h = h, d = sf, segments=100))
    p=hull()(c)
    
    # Holes
    for i in range(m):
        for j in range(n):
            p = p- translate([(i-1)*sf+sd+dh,(j-1)*sf+sd+dh,0])(cylinder(h=h,d=dh,segments=100))
    return(p)

#lego(5,2)
#m=int(input('m : '))
#n=int(input('n : '))
#brick = legol(m,n)
brick = legol(5,2)
r.render(brick)

In [4]:
#2b lab
def bpl(n,h): # Base with Posts
    dl = 4.8
    sd = 3.2/2
    p = cylinder(h=2,d=(dl+2*sd),segments=100) #Positive
    t = float(dl)/2+2*sd
    for i in range(1,n):
        dist = (t+(dl)/2)
        c1 = translate([dist,0,0]) (cylinder(h=2,d=dl+2*sd,segments=100))
        p = p + c1
        t = t+dl+2*sd
    b=hull()(p,c1)
    
    p = cylinder(h=h,d=dl,segments=100) #Positive
    t = dl/2+2*sd
    for i in range(1,n):
        dist = t+(dl/2)
        c1 = translate([dist,0,0]) (cylinder(h=h,d=dl,segments=100))
        p = p + c1
        t = t+dl+2*sd
    
    posts = p+b
    return posts

#h=10
h=int(input('Height : '))
post = bpl(5,h)
r.render(post)

Height : 10


In [5]:
#Fitting
comb = bpl(4,10) + translate([0,0,2.5])(legol(4,1))
r.render(comb)

In [6]:
#Gen 2
f = 25.4 #Inch to mm conversion factor
Nema = [11,17] #Standard
MS = [1.1,1.7] #Motor Size Inches
BHD = [23,31] #Bolt Hole Distance mm
MSD = [5,5] #Motor Shaft Diameter mm
PDia = [22,22] #Pilot Diameter mm
PDep = [2,2] #Pilot Depth mm
BHS = [4,4] #Bolt Hole Size mm

mh1 = 10 #Length of Circular part
mh2 = 8 #Length of Square part
sg = 4 #Length of Side Groove part
c = 1 #Clearance

In [7]:
def NMM(n): #Nema Motor Mount
    if n == 11:
        i = 0
    elif n == 17:
        i = 1
    
    DD = MS[i]*f #Motor Size in mm
    a = BHD[i] #Bolt location
    
    s = translate([0,0,mh2-mh1-PDep[i]])(cylinder(h=mh1,d=PDia[i])) #Circular Base
    s = s + translate([0,0,mh2/2])(cube([DD+sg,DD+sg,mh2+2],True)) # Top
    s = s - translate([0,0,mh2])(cube([DD,DD,4],True)) # Motor Groove
    s = s - translate([0,0,mh2-PDep[i]*2])(cylinder(h=PDep[i],d=PDia[i])) #Pilot
    s = s - translate([0,0,mh2-mh1-PDep[i]])(cylinder(h=mh1+PDep[i],d=MSD[i]+c)) #Shaft
    
    s = s - translate([a/2,a/2,-mh2])(cylinder(h=2*mh2,d=BHS[i])) #Bolt
    s = s - translate([-a/2,a/2,-mh2])(cylinder(h=2*mh2,d=BHS[i])) #Bolt
    s = s - translate([a/2,-a/2,-mh2])(cylinder(h=2*mh2,d=BHS[i])) #Bolt
    s = s - translate([-a/2,-a/2,-mh2])(cylinder(h=2*mh2,d=BHS[i])) #Bolt

    return(s)

n=int(input('Nema Standard : '))
mm = NMM(n)
r.render(mm)

Nema Standard : 17


In [8]:
i=1 # Nema 17
po = ((rotate([-90,0,0]))(bpl(4,10)))
r.render(NMM(Nema[i]) + translate([-(4*6.4)/2,MS[i]*f/2,mh2/2])(po))

In [9]:
#Motor with lego coupling
c = ((rotate([-90,0,0]))(bpl(5,10)))
r.render((translate([-(MS[1]*f/2+sg)/2,MS[1]*f/2+sg,0]))(c))

In [10]:
#Gen 3
def ac(ss): # Adapter Coupling
    sh = 10 #shaft height
    sn  = 0.4 #shaft notch
    a = cylinder (h=sh+2,d=ss+4) - translate([0,0,2])(cylinder (h=sh,d=ss))
    a = a + translate([ss/2-sn,-ss/2,2])(cube([sn,ss,10]))
    #plus shape
    a = a + translate([0,0,-2])(cube([4,2,4],True))
    a = a + translate([0,0,-2])(cube([2,4,4],True))

    r.render(a)
    
ac(MSD[i])

In [11]:
#lego fitting
a = cube([6,40,4])
a = a - translate([3,4,2])(cube([4,2,4],True))
a = a - translate([3,4,2])(cube([2,4,4],True))
r.render(a)